# Algebra Komputerowa 
## Laboratorium 01 - Algorytmy Modularne 

In [1]:
using Oscar
using Random
using StatsBase
using Test
using Primes

In [2]:
factor = Oscar.factor

factor (generic function with 83 methods)

In [3]:
import Primes.primes as primes 
PRIMES = BigInt.(primes(1,1000000))

78498-element Vector{BigInt}:
      2
      3
      5
      7
     11
     13
     17
     19
     23
     29
      ⋮
 999883
 999907
 999917
 999931
 999953
 999959
 999961
 999979
 999983

# Rozwiązywanie układów kongruencji - Chińskie Twierdzenie o Resztach  
## Metoda Lagrange'a

Zaimplementuj rozwiązywanie układów kongruencji funkcję `congruence_system_lagrange(a,m)`, która przyjmuje parametry:  
1) `a` – tablica reszt modulo (dowolne liczby całkowite)  
2) `m` – tablica rozważanych modulo (parami względnie pierwsze liczby)

Funkcja powinna zwracać wartość $x$, taką, że $x \equiv a[i] \pmod{m_i}$.

Postaraj się, żeby funkcja działała dla liczb całkowitych, jak i dla wielomianów nad ciałem z paczki `Oscar.jl`

Przydatne funkcje: 
1) `gcdx(a,b)` - zwraca nwd oraz współczynniki Bezouta.
2) `div(a,b)` - zwraca dokładny iloraz liczb całkowitych. 
3) `mod(a,b)` - zwraca resztę z dzielnia $a$ przez $b$.
4) `prod(T)` - zwraca iloczyn elementów tablicy T. 

In [4]:
function congruence_system_lagrange(a,m)
    k = length(m)     
    #a = mod.(a, m)
    δ = fill(zero(m[1]),k,k)
    for i=1:k
        for j=i+1:k
            nwd, δ[i,j], δ[j,i] = gcdx(m[i],m[j])
            if nwd != 1
                return "error nwd"
            end
        end
    end
    M = prod(m)
    A = []
    for i=1:k
        Δ = prod(δ[j, i] for j in 1:size(δ, 1) if j != i)
        x = div(M,m[i]) 
        push!(A,Δ*x)
    end
    return mod(sum(a .* A),M)  
end

congruence_system_lagrange (generic function with 1 method)

### Testy poprawności działania na liczbach całkowitych

In [5]:
function check_solution(x, a, m)
    return all(i -> mod(x, m[i]) == mod(a[i],m[i]), 1:length(a))
end

check_solution (generic function with 1 method)

In [6]:
function random_integer_congruence(f=100,g=1000,h=1000)
    n = rand(2:f)
    m = PRIMES[sample(1:g,n,replace=false)]
    a = BigInt.(rand(1:h,n))
    return a,m
end

random_integer_congruence (generic function with 4 methods)

In [7]:
function test_congruence(solver)
    for i=1:100
        a,m = random_integer_congruence()
        @assert check_solution(solver(a,m),a,m) "wrong solution! $a, $m"
    end
    println("Test passed!")
end

test_congruence (generic function with 1 method)

In [8]:
test_congruence(congruence_system_lagrange)

Test passed!


## Metoda Przyrostowa Newtona

Zaimplementuj algorytm przyrostowy Newtona. W tym celu najpierw zaimplementuj funkcję pomocniczą `congruence_two(a,m)`, która ma rozwiązywać układ dwóch kongruencji.
Paremetry:
1) `a` - dwuelementowa tablica spodziewanych reszt. 
2) `m` - dwuelementowa tablica (względnie pierwszych) rozważanych modulo.

In [55]:
function congruence_two(a,m)
    _, α, β = gcdx(m[1], m[2])
    r = mod(a[1],m[1])
    b = (a[2] - r)*α 
    s = mod(b,m[2])
    return r + s*m[1]
end

congruence_two (generic function with 1 method)

Zaimplementuj rozwiązywanie układów kongruencji funkcję `congruence_system_newton(m, a)`. Funkcja powinna korzystać z `congruence_two(m,a)`

Parametry, wartości zwracane oraz działanie ma być identyczne z `congruence_system_lagrange(m,a)`.

In [56]:
function congruence_system_newton(a,m)
    Ring = parent(m[1])
    a = Ring.(a)    
    k = length(m)
    M = m[1]
    A = mod(a[1],m[1])
    for i=2:k
        b = congruence_two([A, a[i]],[M,m[i]])
        A = b 
        M *= m[i] 
    end
    return A 
end

congruence_system_newton (generic function with 1 method)

### Testy poprawności działania na liczbach całkowitych

In [57]:
test_congruence(congruence_system_newton)

Test passed!


### Testy wydajnościowe
Porównajmy czasy działania obu metod - zobaczmy, które wypada lepiej.

In [58]:
function solve_all(solver,data)
    for (a,m) in data 
        solver(a,m)
    end
end

solve_all (generic function with 1 method)

In [59]:
n = 5000
data = [random_integer_congruence() for i=1:n]
@time solve_all(congruence_system_lagrange,data)
@time solve_all(congruence_system_newton,data)  

 10.800604 seconds (102.98 M allocations: 2.995 GiB, 22.98% gc time, 0.09% compilation time)
  0.680118 seconds (7.36 M allocations: 238.255 MiB, 28.33% gc time, 1.32% compilation time)


### Testy porównujące maksymalne liczby kongruencji 
Wadą metody Lagrange'a jest też szybki wzrost współczynników $A$. Zobaczmy, że metoda Newtona radzi sobie z 20 kongruencjami, podczas gdy metoda Lagrange'a zwraca zły wynik -  z powodu Integer Overflow

Oczywiście wszystko ma swoją granicę, np. dla 25 kongruencji Newton również zwraca błąd.

In [22]:
n = 250
m = PRIMES[sample(1:5000,n,replace=false)]
a = rand(1:10000,n)
newton = congruence_system_newton(a,m)
check_solution(newton,a,m)

true

Oczywiście, trzeba wiedzieć i pamiętać, że liczba kongruencji to nie jedyna rzecz, z którą możemy przesadzić. Dobierając nawet mało, ale ogromnych liczb pierwszych, szybko okaże się, że wyniki wychodzą błędne. Dodatkowo, skrajne wartości, które tu wskazuje - zostały zaczerpnięte z moich eksperymentów. Być może w innych implementacjach, wyniki mogą być lekko inne.

### Prostsza implementacja rozwiązywania dwóch kongruencji.

Rozważmy układ dwóch kongruencji $x \equiv a \pmod{m_1} \land x \equiv b \pmod{m_2}$. 

Zakładamy, że $m_1,m_2$ są względnie pierwsze - zatem można wyliczyć współczynniki Bezouta: 
$$ \alpha m_1  + \beta m_2 = 1. $$

Zauważmy teraz, że rozwiązaniem układu kongruencji jest po prostu: 
$$ \alpha m_1 b + \beta m_2 a$$
Dlaczego jednak w algorytmie na prezentacji, proponowana była inna implementacja? Chodzi o wielkość pojawiających się czynników. 

Podmieńmy implementacje `congruence_two(a,m)` na taką uproszczoną:

In [60]:
function congruence_two_v2(a,m)
    _, α, β = gcdx(m[1], m[2])
    return mod(α*m[1]*a[2] + β*m[2]*a[1], m[1]*m[2])
end

congruence_two_v2 (generic function with 1 method)

In [61]:
function congruence_system_newton_v2(a,m)
    Ring = parent(m[1])
    a = Ring.(a)    
    k = length(m)
    M = m[1]
    A = mod(a[1],m[1])
    for i=2:k
        b = congruence_two_v2([A, a[i]],[M,m[i]])
        A = b 
        M *= m[i] 
    end
    return A 
end

congruence_system_newton_v2 (generic function with 1 method)

Spróbujmy teraz kilkukrotnie odpalić testy - okaże się, że czasem przechodzą, czasem nie.

In [62]:
n = 50000
data = [random_integer_congruence() for i=1:n]
@time solve_all(congruence_system_newton,data)
@time solve_all(congruence_system_newton_v2,data)  

  7.592501 seconds (73.46 M allocations: 2.320 GiB, 31.13% gc time)
  7.978415 seconds (73.67 M allocations: 2.560 GiB, 20.76% gc time, 1.00% compilation time)


### Testy poprawności działania na wielomianach jednej zmiennej nad ciałem $\mathbb{Q}$.

Dobrze zaimplementowany kod, powinien działać nie tylko dla liczb całkowitych, ale i dla innych pierścieni Euklidesowych - w szczególności wielomianów jednej zmiennej nad ciałem.
W celu przetestowania tego, skorzystamy z paczki `Oscar.jl` z poprzednich laboratoriów.

Nie będziemy tego testować na bardzo ogólnych przypadkach - ograniczymy się do testowania sytuacji, gdy rozważane wielomiany modularne $m_i$ są postaci $x - p_i$ dla pewnego $p_i$. Wtedy rozważane reszty $a_i$ są stopnia zerowego (znaczy są po prostu liczbami).

W takiej sytuacji, problem znalezienia wielomianu $f$, takiego, że $f \equiv a_i \pmod{m_i}$ dla każdego $i$, można równoważnie zapisać jako:
$$ \forall i \quad  f(p_i) = a_i $$
czyli jest to interpolacja wielomianu.

In [25]:
R, x = QQ[:x]

(Univariate polynomial ring in x over QQ, x)

In [40]:
function random_polynomial_congruence()
    n = 100
    a = rand(1:1000,n) 
    x_coords =  sample(1:1000,n,replace=false) 
    m = [x - x_coords[i] for i=1:n]
    return a,m
end

random_polynomial_congruence (generic function with 1 method)

In [41]:
function check_solution_polynomial(f,a,m)
    k = length(m)
    x_cord = [-m[i](0) for i=1:k]
    return all(i -> f(x_cord[i]) == a[i], 1:k) 
end

check_solution_polynomial (generic function with 1 method)

In [42]:
function test_congruence_polynomial(solver)
    for i=1:100
        a,m = random_polynomial_congruence()
        @assert check_solution_polynomial(solver(a,m),a,m) "wrong solution! $a, $m"
    end
    println("Test passed!")
end

test_congruence_polynomial (generic function with 1 method)

In [43]:
test_congruence_polynomial(congruence_system_lagrange)

Test passed!


In [64]:
test_congruence_polynomial(congruence_system_newton_v2)

Test passed!


# Potęgowanie modularne

### Podstawowa metoda szybkiego potęgowania modularnego
Zaimplementuj funkcje `power_modulo(a,n,m)` która przyjmuje na wejściu liczby całkowitę `a,n,m` i zwraca wartość $a^n \pmod{m}$. Wykorzystaj do tego szybkie potęgowanie.

Przydatne funkcje:
1) `digits(n,base)` - zwraca liczbę $n$ w systemie o podstawie $base$

In [65]:
function power_modulo(a,n,m)
    d = digits(n,base=2)
    k = length(d)
    r = 1
    b = mod(a,m)
    for i=1:k
        if(d[i]==1)
            r = mod(r*b,m)
        end 
        b = mod(b^2,m)
    end
    return r 
end

power_modulo (generic function with 1 method)

In [66]:
power_modulo(49,19,11)

9

### Testy poprawności działania potęgowania modularnego

In [73]:
function test_power_modulo()
    for i=1:100
        a = rand(BigInt(1):BigInt(1_000_000))
        n = rand(BigInt(1):BigInt(1_000_000))
        m = rand(BigInt(1):BigInt(1_000_000))
        @assert power_modulo(a,n,m) == powermod(a,n,m) "error for a=$a, n=$n, m=$m"
    end
    println("test passed!")
end

test_power_modulo (generic function with 1 method)

In [74]:
test_power_modulo()

test passed!


### Funkcja $\varphi$ Eulera
Zaimplementuj funkckję $\phi(n)$ przyjmującą na wejściu liczbę naturalną $n$ i zwracającą na wyjściu wartość funkcji Eulera $\varphi(n)$. 
Przydatne funkcje:
1) `factor(n)` - zwraca faktoryzację na czynniki pierwsze liczby $n$. 
Uwaga - w Paczcze `Oscar` i `Primes` osobno zdefinowane funkcje `factor()`. Żeby ominąć niejednoznaczności można pisać `Oscar.factor()` lub `Primes.factor()`. Nie ma to szczególnego znaczenia, ale jak testowałem dla bardzo dużych liczb, to `Oscar.factor` wypada lekko szybciej. 
Możemy też zapisać po prostu:

In [75]:
factor = Oscar.factor 

factor (generic function with 83 methods)

In [76]:
function ϕ(n) 
    res = 1
    for (p, α) in factor(n)
        res*=(p^α - p^(α -1))
    end
    return res 
end

ϕ (generic function with 1 method)

### Testy poprawności działania

In [77]:
function test_phi_func()
    for i=1:1000
        n = rand(1:10^6)
        @assert ϕ(n) == totient(n)
    end
    println("Test passed!")
end

test_phi_func (generic function with 1 method)

In [78]:
test_phi_func()

Test passed!


### Szybkie Potęgowanie Modulo wykorzystując funkcje $\varphi$ Eulera.
Zaimplementuj funkcje `power_modulo_euler(a,n,m)`, która działa analogicznie do funkcji `power_modulo(n)`, wykorzystując dodatkowo twierdzenie Eulera.

In [81]:
function power_modulo_euler(a,n,m)
    d = gcd(a,m)
    p = 1 
    if(d>1)
        a = div(a,d)
        p = power_modulo(d,n,m)
    end
    n = mod(n, ϕ(m))
    return mod(p*power_modulo(a,n,m),m)
end

power_modulo_euler (generic function with 1 method)

### Testy poprawności działania

In [82]:
function test_power_modulo_euler()
    for i=1:100
        a = rand(1:100000000)
        n = rand(1:100000000)
        m = rand(1:100000000)
        @assert power_modulo_euler(a,n,m) == powermod(a,n,m) "error for a=$a, n=$n, m=$m"
    end
    println("test passed!")
end

test_power_modulo_euler (generic function with 1 method)

In [83]:
test_power_modulo_euler()

test passed!


### Porównanie wydajności - ile "oszczędza" nam Twierdzenie Eulera. 

In [84]:
function solve_all_power(solver, data)
    for (a,n,m) in data 
        solver(a,n,m)
    end
end

solve_all_power (generic function with 1 method)

In [86]:
n = 1_000_000
data = [[rand(1:10^9), rand(1:10^9), rand(1:10^9)] for j=1:n]
@time solve_all_power(power_modulo,data)
@time solve_all_power(power_modulo_euler,data)
@time solve_all_power(powermod,data)

  0.971123 seconds (2.00 M allocations: 278.966 MiB, 18.81% gc time)
  7.099256 seconds (33.67 M allocations: 1.448 GiB, 7.09% gc time)
  0.782453 seconds


Jak widać.... wyniki sporo poniżej oczekiwań. Liczenie funkcji Eulera jest widocznie na tyle kosztowne, że nie opłaca się tego robić, a przynajmniej nie w ten sposób. Istnieją inne metody wyznaczania jej, ale nie wiadomo, czy wciąż się to opłaca. 

Co ciekawe natknąłem sie na taki post na [forum Julii](https://discourse.julialang.org/t/optimizing-powermod/115936)
można znaleźć tam taką implementację funkcji:

In [87]:
function my_powermod(a::Int64, b::Int64, c::Int64)
    @assert b >= 0
    result = one(Int128)
    a_raised_to_powers_of_two = convert(Int128, a)
    while b != 0
        if isodd(b)
            result *= a_raised_to_powers_of_two
            result = result % c;
        end
        b = b >>> 1
        a_raised_to_powers_of_two = a_raised_to_powers_of_two^2 % c
    end
    convert(Int64, result)
end

my_powermod (generic function with 1 method)

Zauważmy, że jest to implementacja `power_modulo()`. Po prostu, zamiast używać sztucznie `digits()` sprawdzana jest parzystość i na tej podstawie określane są kolejne bity.

Interesujące jest to, że ta funkcja jest szybsza niż funkcja wbudowana `powermod`! 

In [88]:
n = 1_000_000
data = [[rand(1:10^9), rand(1:10^9), rand(1:10^9)] for j=1:n]
@time solve_all_power(powermod,data)
@time solve_all_power(my_powermod,data)

  0.786682 seconds
  0.745398 seconds (14.93 k allocations: 761.383 KiB, 4.50% compilation time)


# Liczenie wyznacznika Macierzy całkowitoliczbowej.

Rozważmy macierz $M$ o elementach całkowitych. Jej wyznacznik $\det(M)$ należy również do liczb całkowitych. 
Klasyczne algorytmy liczenia wyznacznika, wykorzystują dzielenie, przez co implementacja algorytmu liczenia wyznacznika, może nam zwrócić niedokładną wartości z powodu nieprecyzyjnej reprezentacji liczb zmiennoprzecinkowych.

### Nie trzeba daleko szukać:

In [90]:
M = [3 1 -2;
     2 -3 -1;
    1 3 0]
println("typ elementu macierzy: $(typeof(M[1,1]))")
println("wyznacznik macierzy: $(det(M))") 

typ elementu macierzy: Int64
wyznacznik macierzy: -9.999999999999998


### Wyznacznik tej macierzy, jest równy -10. Mimo to, funkcja biblioteczna zwraca wynik niecałkowity...

Można by sobie poradzić z tym na wiele sposobów. Najprostszym pomysłem, jest reprezentacja liczb wymiernych jako ułamki (licznik/mianownik). W ten sposób moglibyśmy uzyskać dokładny wynik, ale działanie na ułamkach w ten sposób jest drogie - mianowniki zwykle bardzo szybko rosną. 

Sprytniejszym pomysłem jest zastosowanie *chińskiego twierdzenia o resztach*. 
Jeżeli tylko ograniczymy wartośc wyznacznika, można policzyć wyznacznik modulo wiele liczb pierwszych, a następnie złączyć wyniki modulo małe liczby pierwsze do modulo na tyle dużego - że otrzymamy jednoznaczny wynik.

## Nierówność Hadamarda 

Ograniczenie górne na wartość wyznacznika znane jest jako nierówność Hadamarda. To ją wykorzystamy. 

Zaimplementuj funkcję `hadamard_bounding(M)`, która przyjmuję na wejściu macierz $M$ i zwraca ograniczenie górne (całkowitoliczbowe) na jej wyznacznik.

In [221]:
function hadamard_bounding(M)
    entries = collect(M)                    
    m = maximum(abs.(entries))             
    n = BigInt(size(M, 1))                           
    return ceil(typeof(M[1,1]),m^n * n^(n/2))
end

hadamard_bounding (generic function with 1 method)

### Testy poprawności działania 

(na bardzo pojedynczych przykładach)

In [92]:
function test_hadamard()
    M1 = [3 1 -2; 2 -3 -1;1 3 0]
    M2 = reshape(1:16, 4, 4)
    M3 = [87  42  73  10; 25  11  99  58; 67  30  14  77; 92  51  60  18 ]
    @assert hadamard_bounding(M1) == 141 "error in M1"
    @assert hadamard_bounding(M2) == 1048576 "error in M2"
    @assert hadamard_bounding(M3) == 1536953616 "error in M3"
    println("Test passed!")
end

test_hadamard (generic function with 1 method)

In [93]:
test_hadamard()

Test passed!


### Odwrotność elementu w ciele $\mathbb{Z}_p$ 
Zaimplementuj funkcje `inverse(a,p)`, która przyjmuje zwraca odwrotność liczby $a$ względem mnożenia w ciele $\mathbb{Z}_p$.

In [94]:
function inverse(a,p)
    return mod(gcdx(a,p)[2],p)
end

inverse (generic function with 1 method)

### Eliminacja Gaussa Modulo p
Zaimplementuj funkcję `gauss_modulo(M,n,p)`, która liczy wyznacznik macierzy $M$ o rozmiarze $n \times n$ modulo $p$.

In [ ]:
function gauss_modulo(M,n,p)
    A = copy(M)
    A = mod.(A,p)
    det = BigInt(1) 

    for k in 1:n-1
        max_row = k
        max_val = abs(A[k, k])
        
        for i in k+1:n
            if abs(A[i, k]) > max_val
                max_val = abs(A[i, k])
                max_row = i
            end
        end
        
        if max_row != k
            A[[k, max_row], :] = A[[max_row, k], :]
            det *=(-1)
        end
        
        if A[k, k] == 0
            return 0 
        end
        
        
        for i in k+1:n
            factor = mod(A[i, k] *inverse(A[k, k],p),p)
            A[i, k:end] = mod.(A[i, k:end] - factor * A[k, k:end],p)
        end
    end
    
    for i in 1:n
        det = mod(det*A[i, i],p)
    end
    
    return det
end

gauss_modulo (generic function with 1 method)

In [98]:
gauss_modulo(M,3,7)

4

### Liczenie wyznacznika Macierzy Całkowitoliczbowej.
Zaimplementuj funkcję `modular_determinant(M)` która liczy wyznacznik macierzy całkowitoliczbowej $M$ wykorzystując obliczenia modulo wiele liczb pierwszych $p$.

In [166]:
function get_primes(BOUND)
    P = 1
    i = 0
    while P <= BOUND
        i+=1
        P*=PRIMES[i]
    end
    return PRIMES[1:i],P
end

get_primes (generic function with 1 method)

In [251]:
function modular_determinant(M)
    n = size(M,1)
    
    BOUND = 2*hadamard_bounding(M)
    used_primes,P = get_primes(BOUND)
    d = BigInt[]
    for p in used_primes
        push!(d, gauss_modulo(M,n,p))
    end
    det =  congruence_system_lagrange(d,used_primes)
    if det > P/2
        det-=P 
    end 
    return det
end

modular_determinant (generic function with 1 method)

In [168]:
M2 = A

3×3 Matrix{Int64}:
 3   1  -2
 2  -3  -1
 1   3   0

In [169]:
det(M2)
M3 = matrix(ZZ,M2)
det(M3)

-10

In [171]:
modular_determinant(M2)

-10

### Testy poprawności działania

In [172]:
function test_modulo_determinant()
    for i=1:100
        n = rand(1:15)
        M = rand(BigInt(-10):BigInt(10),n,n)
        x = modular_determinant(M)
        y = det(M) 
        #y = det(matrix(ZZ,M)) 
        @assert isapprox(x,y) "$x, $y"
    end
    println("Test passed!")
end

test_modulo_determinant (generic function with 1 method)

In [173]:
test_modulo_determinant()

Test passed!


### Testy wydajności

In [179]:
function gauss_determinant(M)
    n = size(M,1)
    A = Rational{BigInt}.(copy(M))
    det = 1.0  

    for k in 1:n-1
        # Pivot selection for numerical stability
        max_row = k
        max_val = abs(A[k, k])
        for i in k+1:n
            if abs(A[i, k]) > max_val
                max_val = abs(A[i, k])
                max_row = i
            end
        end

        # Swap rows if needed
        if max_row != k
            A[[k, max_row], :] = A[[max_row, k], :]
            det *= -1
        end

        # If pivot is zero, determinant is zero
        if A[k, k] == 0
            return 0.0
        end

        # Eliminate below pivot
        for i in k+1:n
            factor = A[i, k] / A[k, k]
            A[i, k:end] .-= factor * A[k, k:end]
        end
    end

    # Multiply diagonal entries
    for i in 1:n
        det *= A[i, i]
    end

    return det
end


gauss_determinant (generic function with 1 method)

In [180]:
gauss_determinant(M2)

-10.0

In [239]:
function random_matrix()
    n = rand(10:20)
    M = rand(BigInt(-200):BigInt(200),n,n)
    return M   
end

random_matrix (generic function with 1 method)

In [243]:
M = random_matrix()
@time  a = modular_determinant(M)
#@time  b = gauss_determinant(M)
#@assert isapprox(a,b) 

InexactError: InexactError: Int64(66914519259363915502150)

In [224]:
size(M,1)

198

In [223]:
@time det(M)

  4.683324 seconds (28.09 M allocations: 1.505 GiB, 18.57% gc time)


223768933078465267887612978145925892603670041692206583280341743916792491172255013790246452410843252342163723432268015218001226439670605789845821783874598929042333349223790450351354093601677509336740440866222313043837617939012054178573641797349198883963103102152903550284809078129906219233290378334810433930023087068233592784918758177901009564517826225539366434642450284148113290666780575950683107411494320600170372870085870907777241220483426232147990192230268787569459722292199316105723570171834408741509650873562746728662479526788419934265769829104333152254361032789188506747822325077877890902

In [191]:
size(M,1)
M
a 
b

-3.180147892668926040680504836112473011273447338626626356531665901647737317327968e+250

In [37]:
function gauss_finite_field(A)
    n = size(A, 1)
    F = base_ring(A)
    p = characteristic(F)
    det = one(F)
    swaps = 0
    
    for k in 1:n-1
        pivot_row = k
        while pivot_row ≤ n && iszero(A[pivot_row, k])
            pivot_row += 1
        end
        
        if pivot_row > n
            return zero(F)
        end
        
        if pivot_row != k
            A[[k, pivot_row], :] = A[[pivot_row, k], :]
            swaps += 1
        end
        
        pivot = A[k, k]
        for i in k+1:n
            factor = A[i, k] * inv(pivot)
            for j in k:n
                A[i, j] = A[i, j] - factor * A[k, j]
            end
        end
    end
    
    for i in 1:n
        det *= A[i, i]
    end
    
    if isodd(swaps)
        det = -det
    end
    
    return det
end

gauss_finite_field (generic function with 1 method)

In [41]:
function modular_determinant_finite_field(M)
    n = size(M,1)
    BOUND = 2*hadamard_bounding(M)
    println("bound $BOUND")
    P = 1
    i = 0
    while P <= BOUND
        i+=1
        P*=PRIMES[i]
    end
    used_primes =PRIMES[1:i]
    println("big P $P")
    println("miau $used_primes")
    d = []
    for p in used_primes
        A = matrix(GF(p),M)
        push!(d, gauss_finite_field(A))
    end
    println("dets $d")
    d =[lift(ZZ,x) for x in d]
    det =  congruence_system_lagrange(used_primes,d)
    print("wow $d")
    if det > P/2
        det-=P 
    end 
    return det
end

modular_determinant_finite_field (generic function with 1 method)

In [42]:
M2

3×3 Matrix{Int64}:
 3   1  -2
 2  -3  -1
 1   3   0

In [43]:
modular_determinant_finite_field(M2)

bound 282
big P 2310
miau [2, 3, 5, 7, 11]
dets Any[0, 2, 0, 4, 1]
wow ZZRingElem[0, 2, 0, 4, 1]

-10

In [161]:
A = [3   1  -2;
 2  -3  -1;
 1   3   0;]

3×3 Matrix{Int64}:
 3   1  -2
 2  -3  -1
 1   3   0

In [ ]:
F = GF(7)
M = matrix(F, [2 -1 0; -1 2 -1; 0 -1 2])
M2 = deepcopy(M)

deter = gauss_elimination_det_GF(M2)
println("The determinant is: ", deter, " real", det(M))

In [193]:
BigInt(nextprime(2^63))

2

In [205]:
p = nextprime(BigInt(2)^63)
result = BigInt(p)

9223372036854775837

In [237]:
function get_big_primes(k)
    BIGPRIMES = BigInt[]
    p = nextprime(BigInt(2)^100)
    for i=1:k 
        push!(BIGPRIMES,p)
        p = nextprime(p+1)
    end 
    return BIGPRIMES
end

get_big_primes (generic function with 1 method)

In [238]:
BIGPRIMES = get_big_primes(1000)

1000-element Vector{BigInt}:
 1267650600228229401496703205653
 1267650600228229401496703205707
 1267650600228229401496703205823
 1267650600228229401496703205901
 1267650600228229401496703205953
 1267650600228229401496703206003
 1267650600228229401496703206019
 1267650600228229401496703206187
 1267650600228229401496703206273
 1267650600228229401496703206297
                               ⋮
 1267650600228229401496703273521
 1267650600228229401496703273549
 1267650600228229401496703273713
 1267650600228229401496703273929
 1267650600228229401496703273933
 1267650600228229401496703273941
 1267650600228229401496703274017
 1267650600228229401496703274083
 1267650600228229401496703274119

In [211]:
function get_primes(BOUND)
    P = 1
    i = 0
    while P <= BOUND
        i+=1
        P*= BIGPRIMES[i]
    end
    return BIGPRIMES[1:i],P
end

get_primes (generic function with 1 method)

In [226]:
nextprime(BigInt(2)^250)

1809251394333065553493296640760748560207343510400633813116524750123642650649

In [241]:
(BigInt(3)^59 + 4 )% BigInt(2)^100

14130386091738734504764811071